# Mount drive

In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

!ls /content/gdrive/My\ Drive

Mounted at /content/gdrive
 aimas2020
'Automatic Generation of Topic Labels.gslides'
'Colab Notebooks'
 cvdl2020
 iir_book.pdf
 ir_final
'Medical AI'
'Paper Slides'
 Q56094077
 res18_diabete_noaug.pth
'Towards Better Text Understanding and Retrieval through Kernel Entity Saliency Modeling.gslides'
 tsai.ipynb
 獎助學金
 申請資料


In [ ]:
# !unzip /content/gdrive/MyDrive/Q56094077/snrs/hw1_0319/hw1_data.zip -d /content/gdrive/MyDrive/Q56094077/snrs/hw1_0319

# Import Library

In [1]:
import os

import torch
import torch.nn as nn

import pandas as pd
import numpy as np
import json

from tqdm import tqdm
from datetime import datetime

In [2]:
import torch_geometric
from torch_geometric.data import Data, DataLoader
import torch_geometric.utils as utils

# Setting

In [3]:
class Setting:
    _root = os.getcwd()

    _data = os.path.join(_root, "hw1_data")

    data_synthetic = os.path.join(_data, "Synthetic", "5000")
    data_youtube = os.path.join(_data, "youtube")
    
    
     # Create dir for train/test
    date_time = datetime.strftime(datetime.now(), "%Y-%m-%d %H-%M")
    root = os.path.join(_root,  date_time)
    if os.path.exists(root):
        pass
    else:
        os.makedirs(root)


    ### Save plt info
    train_info_p = os.path.join(root, "train.json")
    val_info_p = os.path.join(root, "valid.json")
    test_info_p = os.path.join(root, "test.json")

    ### Save plt img
    result_plt_p = os.path.join(root, "train_plt.png")
    test_plt_p = os.path.join(root, "test_plt.png")
    sum_box_p = os.path.join(root, "sum_box.png")
        
        
    ### Data split
    data_split = os.path.join(_root, "split.json")
    

    # Setting of training
    device = torch.device("cuda:1" if torch.cuda.is_available() else "cpu")

    epochs = 10000
    batch = 8

    c = 3
    depth = 5
    p = 128 # embedding dimension of hidden state
    q = int(p/2)

    save_model = os.path.join(root, "weight.pth")
#     criterion = torch.nn.BCEWithLogitsLoss(reduction="sum")
    criterion = nn.BCELoss()


In [4]:
setting = Setting()

# Dataset

## Data

- data.x	节点特征，维度是[num_nodes, num_node_features]。
- data.edge_index	维度是[2, num_edges]，描述图中节点的关联关系，每一列对应的两个元素，分别是边的起点和重点。数据类型是torch.long。需要注意的是，data.edge_index是定义边的节点的张量（tensor），而不是节点的列表（list）。
- data.edge_attr	边的特征矩阵，维度是[num_edges, num_edge_features]
- data.y	训练目标（维度可以是任意的）。对于节点相关的任务，维度为[num_nodes, *]；对于图相关的任务，维度为[1,*]。
- data.position	节点位置矩阵（Node position matrix），维度为[num_nodes, num_dimensions]。

- [Learning to Identify High Betweenness Centrality Nodes from
Scratch: A Novel Graph Neural Network Approach](https://arxiv.org/pdf/1905.10418.pdf)
- node initial feature = [$(d_v), 1, 1]

### Load all data

In [5]:
synthetic = []
between = []
for f in os.listdir(setting.data_synthetic):
    if "score" in f:
        # ground truth of betweenness centrality
        p = os.path.join(setting.data_synthetic, f)
        between.append(p)
        pass
    else:
        p = os.path.join(setting.data_synthetic, f)
        synthetic.append(p)

between.sort()
synthetic.sort()

### Split Data

In [6]:
from sklearn.model_selection import train_test_split


def split_data(path, replace=False):
    if os.path.exists(path) and replace:
        pass
    else:
        X_train, X_test, y_train, y_test = train_test_split(synthetic, between, test_size=0.2)
        X_train, X_valid, y_train, y_valid = train_test_split(X_train, y_train, test_size=0.15)

        split = {
            "X_train": X_train,
            "X_valid": X_valid,
            "X_test": X_test,
            "y_train": y_train,
            "y_valid": y_valid,
            "y_test": y_test
        }

        with open(path, 'w') as f:
            json.dump(split, f)
            
split_data(path = setting.data_split)

### Cvt 2 dataloader

In [7]:
def to_dataloader(x, y, batch):
    
    data_list = []
    for index, f in enumerate(x):
        edge_index = torch_geometric.io.read_txt_array(f, dtype=torch.long)
        edge_index = edge_index.t().contiguous()
        edge_index = utils.to_undirected(edge_index)

        row, col = edge_index  
        deg = utils.degree(col) # must use col to get degree, why?
        deg = deg.numpy()  

        vertice = []
        for d in deg:
            vertice.append([d, 1, 1])
        vertice = np.array(vertice, dtype=np.float)
        vertice = torch.from_numpy(vertice)

        ### between centrality
        bcs = []
        bc = torch_geometric.io.read_txt_array(y[index], dtype=torch.double)
        bc = bc.t().contiguous()
        row, col = bc
        bc = col
        bc = bc.numpy()
        for b in bc:
            bcs.append([b])

        data = Data(x=vertice, edge_index=edge_index, y=bcs)
        data_list.append(data)

    loader = DataLoader(data_list, batch_size=batch)
    return loader

#### Read data

In [8]:
f = open(setting.data_split, 'r')
data_split = json.load(f)

X_train = data_split["X_train"]
X_valid = data_split["X_valid"]
X_test = data_split["X_test"]
y_train = data_split["y_train"]
y_valid = data_split["y_valid"]
y_test = data_split["y_test"]

f.close()

#### Cvt

In [9]:
train_loader = to_dataloader(X_train, y_train, batch=1)
valid_loader = to_dataloader(X_valid, y_valid, batch=1)
test_loader = to_dataloader(X_test, y_test, batch=1)

#### Use all data

In [10]:
all_loader = to_dataloader(synthetic, between, batch=1)

# Model

In [11]:
from torch_geometric.nn import MessagePassing
import torch.nn.functional as F
from torch_geometric.typing import Adj, OptTensor

In [12]:
class Encoder(MessagePassing):
    def __init__(self, c, p, num_layers, device, aggr="add"):
        super(Encoder, self).__init__(aggr=aggr)
        
        self.num_layers = num_layers
        self.w_0 = nn.Linear(in_features=c, out_features=p).double()
        
        self.relu = nn.ReLU(inplace=True)
        self.rnn = nn.GRUCell(p, p).double()

        self.device = device

    def forward(self, data):
        
        x, edge_index = data.x, data.edge_index
        
        
        # compute dgree
        row, col = edge_index
        deg = utils.degree(col, x.size(0), dtype=x.dtype)
        deg = torch.add(deg, 1)
        deg_inv_sqrt = torch.pow(deg, -0.5)
        norm = deg_inv_sqrt[row] * deg_inv_sqrt[col]
        
        
        # h_0 = x

        # h_1
        x = self.w_0(x)
        x = self.relu(x)
        x = F.normalize(x, p=2, dim=1)
        
        h_s = [x]
        
        for i in range(self.num_layers-1):
            # internally calls the message(), aggregate() and update() functions
            m = self.propagate(edge_index, x=x, norm=norm)
            x = self.rnn(m, x)
            x = F.normalize(x, p=2, dim=1) 
           
            h_s.append(x)
        
        h_s = torch.stack(h_s, dim=-1)
        # Use torch.max to replace max_pooling
        z, _ = torch.max(h_s, dim=-1)
        # z = global_max_pool(h_s, torch.tensor([0], dtype=torch.long).to(self.device))
        
        return z

    def message(self, x_j, norm: OptTensor):
        return x_j if norm is None else norm.view(-1, 1) * x_j
    

In [13]:
class Decoder(nn.Module):
    def __init__(self, p, q):
        
        super().__init__()
        
        self.w_4 = nn.Linear(in_features=p, out_features=q).double()
        self.w_5 = nn.Linear(in_features=q, out_features=1).double()
        
        self.relu = nn.ReLU(inplace=True)
        
    def forward(self, z):
        z = self.w_4(z)
        z = self.relu(z)
        z = self.w_5(z)
        
        return z

In [14]:
class DrBC(nn.Module):
    def __init__(self, encoder_params, decoder_params):
        super().__init__()
        
        self.encoder = Encoder(**encoder_params)
        self.decoder = Decoder(**decoder_params)
        
        
    def forward(self, data):
        
        z = self.encoder(data)
        
        return self.decoder(z)

In [15]:
DrBC(
    encoder_params = dict(
        c = setting.c,
        p = setting.p,
        num_layers = setting.depth,
        device = setting.device
    ),
    decoder_params = dict(
        p = setting.p,
        q = setting.q
    )
)

DrBC(
  (encoder): Encoder(
    (w_0): Linear(in_features=3, out_features=128, bias=True)
    (relu): ReLU(inplace=True)
    (rnn): GRUCell(128, 128)
  )
  (decoder): Decoder(
    (w_4): Linear(in_features=128, out_features=64, bias=True)
    (w_5): Linear(in_features=64, out_features=1, bias=True)
    (relu): ReLU(inplace=True)
  )
)

# Load model

In [16]:
def load_checkpoint(filepath, device, **params):

    model = DrBC(
                encoder_params = dict(
                    c = setting.c,
                    p = setting.p,
                    num_layers = setting.depth,
                    device = setting.device
                ),
                decoder_params = dict(
                    p = setting.p,
                    q = setting.q
                )
            ).to(setting.device)
    
    if os.path.exists(filepath):
        print("pretrained finded")
        checkpoint = torch.load(filepath)
        model.load_state_dict(checkpoint['model_stat'])
        optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)
        optimizer.load_state_dict(checkpoint['optimizer_stat'])

    else:
        print("use a new optimizer")
        optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)

    return model, optimizer

In [17]:
model, optimizer = load_checkpoint(
                    setting.save_model,
                    setting.device)

use a new optimizer


In [18]:
model

DrBC(
  (encoder): Encoder(
    (w_0): Linear(in_features=3, out_features=128, bias=True)
    (relu): ReLU(inplace=True)
    (rnn): GRUCell(128, 128)
  )
  (decoder): Decoder(
    (w_4): Linear(in_features=128, out_features=64, bias=True)
    (w_5): Linear(in_features=64, out_features=1, bias=True)
    (relu): ReLU(inplace=True)
  )
)

# Evaluation

In [19]:
def top_n(model, data, k):

    with torch.no_grad():
        
#         cnt = 0
        intsec = []
        for d in tqdm(data):
            
            d = d.to(setting.device)
            
            bc_gt = d.y
            bc_gt = np.array(bc_gt)
            bc_gt = torch.from_numpy(bc_gt)
            bc_gt = torch.reshape(bc_gt, (-1, ))

            bc_pr = model(d)

            bc_gt = bc_gt.squeeze().to(setting.device)
            bc_pr = bc_pr.squeeze()

            gt_value, gt_indice = torch.topk(bc_gt, k)
            pr_value, pr_indice = torch.topk(bc_pr, k)

            gt_indice = set(gt_indice.cpu().numpy())
            pr_indice = set(pr_indice.cpu().numpy())

            intersect = len(gt_indice & pr_indice)
            intsec.append(intersect/(bc_gt.shape[0] * k / 100))
#             cnt += d.num_graphs

        return intsec

# Train

In [20]:
train_info = {
    "bce": [],
    "top1": [],
    "top5": [],
    "top10": []
}

In [ ]:
model.train()

top_10 = 0.0

for epoch in range(setting.epochs):
    
    bce_loss = 0.0
    graph_cnt = 0
    for data in tqdm(all_loader):
        
        optimizer.zero_grad()
        
        data = data.to(setting.device)
        bc_pr = model(data)
        
        bc_gt = data.y
        bc_gt = np.array(bc_gt)

        bc_gt = torch.from_numpy(bc_gt)
        bc_gt = torch.reshape(bc_gt, (-1, ))

        ### random sample 5|V| nodes
        src = (torch.rand(25000) * 4999).long()
        det = (torch.rand(25000) * 4999).long()
        for b in range(len(data.batch)//5000-1):
            src = torch.cat((src, (torch.rand(25000) * 4999).long()+(b+1)*5000))
            det = torch.cat((det, (torch.rand(25000) * 4999).long()+(b+1)*5000))
        
        y_gt = (bc_gt[det] - bc_gt[src]).squeeze().to(setting.device)
        y_pr = (bc_pr[det] - bc_pr[src]).squeeze()

#         print(torch.max(y_gt), torch.min(y_gt))
#         print(torch.max(y_pr), torch.min(y_pr))
        
        y_gt = nn.Sigmoid()(y_gt)
        y_pr = nn.Sigmoid()(y_pr)
        
        ### Map gt to 0 and 1
        y_gt = torch.where(y_gt >= 0.5, torch.ones_like(y_gt), y_gt)
        y_gt = torch.where(y_gt < 0.5, torch.zeros_like(y_gt), y_gt)
        
#         print(torch.max(y_gt), torch.min(y_gt))
#         print(torch.max(y_pr), torch.min(y_pr))
#         break
        

        loss = setting.criterion(y_pr, y_gt)
        bce_loss += loss.item()
        graph_cnt += data.num_graphs
    
        loss.backward()
        optimizer.step()

    
    top1 = top_n(model, all_loader, k=1)
    top5 = top_n(model, all_loader, k=5)
    top10 = top_n(model, all_loader, k=10)
    l = bce_loss/graph_cnt
    
    print("Epoch = {}, loss = {}, top 1 = {}, top 5 = {}, top 10 = {}".format(epoch+1, l, 
                                                                              sum(top1)/len(top1), 
                                                                              sum(top5)/len(top5), 
                                                                              sum(top10)/len(top10)))
    
    train_info["bce"].append(l)
    train_info["top1"].append(top1)
    train_info["top5"].append(top5)
    train_info["top10"].append(top10)
    with open(setting.train_info_p, 'w') as f:
        json.dump(train_info, f)

        
    if sum(top10)/len(top10) > top_10:
        checkpoint = {
            'model_stat': model.state_dict(),
            'optimizer_stat': optimizer.state_dict(),
        }
        torch.save(checkpoint, setting.save_model)
        
        print("save model to {}".format(setting.save_model))
        
        top_10 = sum(top10)/len(top10)

 10%|█         | 3/30 [00:00<00:01, 20.84it/s]

Epoch = 1, loss = 0.6908091733374963, top 1 = 0.0, top 5 = 0.001733333333333334, top 10 = 0.004866666666666669
save model to /home/ai2019/ne6081064/june/mlg/snrs_hw1/2021-03-13 11-52/weight.pth


 10%|█         | 3/30 [00:00<00:01, 21.20it/s]

Epoch = 2, loss = 0.6840661298497747, top 1 = 0.0, top 5 = 0.0008, top 10 = 0.0014666666666666671


 10%|█         | 3/30 [00:00<00:01, 20.84it/s]

...

 10%|█         | 3/30 [00:00<00:01, 20.91it/s]

Epoch = 246, loss = 0.47799384241787923, top 1 = 0.0006666666666666666, top 5 = 0.004266666666666668, top 10 = 0.009266666666666671


  0%|          | 0/30 [00:00<?, ?it/s]

Epoch = 247, loss = 0.479481287833539, top 1 = 0.0006666666666666666, top 5 = 0.005200000000000002, top 10 = 0.01093333333333334


 70%|███████   | 21/30 [00:00<00:00, 26.48it/s]

In [ ]:
# test fit model
predict = model(data.to(device))
predict.shape


### Sampling nodes

In our experiments, we randomly sample 5|V | source nodes and 5|V |
target nodes with replacement

In [274]:
picked = (torch.rand(25000, 2) * 4999).long()
for b in range(batch-1):
    picked = torch.stack((picked, (torch.rand(25000, 2) * 4999).long()))
    
picked.shape

torch.Size([25000, 2])

In [41]:
for data in loader:
    print(data)

Batch(batch=[40000], edge_index=[2, 319718], x=[40000, 3], y=[8])
Batch(batch=[40000], edge_index=[2, 319710], x=[40000, 3], y=[8])
Batch(batch=[40000], edge_index=[2, 319722], x=[40000, 3], y=[8])
Batch(batch=[30000], edge_index=[2, 239798], x=[30000, 3], y=[6])


In [45]:
loader(0)

TypeError: 'DataLoader' object is not callable

In [192]:
(loader[0].y-loader.y[1]).shape

AttributeError: 'DataLoader' object has no attribute 'y'